In [1]:
!pip install selenium beautifulsoup4 pandas tenacity

In [1]:
import time
import pandas as pd
from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.options import Options
from selenium.common.exceptions import NoSuchElementException, ElementClickInterceptedException
from tenacity import retry, stop_after_attempt, wait_fixed

In [2]:
def init_driver():
    options = Options()
    options.add_argument("--headless")
    options.add_argument("--no-sandbox")
    options.add_argument("--disable-dev-shm-usage")
    return webdriver.Chrome(options=options)

In [3]:
@retry(stop=stop_after_attempt(3), wait=wait_fixed(2))
def get_page_source(driver, url):
    driver.get(url)
    time.sleep(2)
    return driver.page_source

In [4]:
SECTIONS = [
    "https://www.aljazeera.com/news",
    "https://www.aljazeera.com/middle-east",
    "https://www.aljazeera.com/africa",
    "https://www.aljazeera.com/asia",
    "https://www.aljazeera.com/europe",
    "https://www.aljazeera.com/us-canada",
    "https://www.aljazeera.com/business-economy",
    "https://www.aljazeera.com/sports",
    "https://www.aljazeera.com/features",
    "https://www.aljazeera.com/opinions"
]

In [5]:
def click_show_more_until_end(driver):
    while True:
        try:
            show_more = driver.find_element(By.CSS_SELECTOR, "button.show-more-button")
            driver.execute_script("arguments[0].scrollIntoView(true);", show_more)
            time.sleep(1)
            show_more.click()
            time.sleep(2)
        except (NoSuchElementException, ElementClickInterceptedException):
            break

In [6]:
def extract_article_links_dynamic(driver, section_url):
    print(f"🔍 Visiting: {section_url}")
    driver.get(section_url)
    time.sleep(2)
    click_show_more_until_end(driver)
    soup = BeautifulSoup(driver.page_source, 'html.parser')
    links = set()
    for tag in soup.select('a.u-clickable-card__link'):
        href = tag.get("href")
        if href and href.startswith("/"):
            links.add("https://www.aljazeera.com" + href)
    return list(links)

In [7]:
def extract_article_data(driver, url):
    try:
        driver.get(url)
        time.sleep(2)
        soup = BeautifulSoup(driver.page_source, "html.parser")
    except Exception as e:
        print(f"❌ Failed to scrape {url}: {e}")
        return {
            "Title": "N/A",
            "Url": url,
            "Date": "N/A",
            "Description": "N/A"
        }

    # Extract title
    title_tag = soup.find("h1")
    title = title_tag.get_text(strip=True) if title_tag else "N/A"

    # Extract date
    date_tag = soup.find("time")
    date = date_tag.get("datetime") if date_tag else "N/A"

    # Extract article content
    body = soup.find("div", class_="wysiwyg") or soup.find("div", {"data-component": "text-block"})
    if body:
        paragraphs = body.find_all("p")
        description = " ".join(p.get_text(strip=True) for p in paragraphs)
    else:
        description = "N/A"

    return {
        "Title": title.strip(),
        "Url": url.strip(),
        "Date": date.strip(),
        "Description": description.strip()
    }

In [8]:
def save_to_csv(data, filename="aljazeera_articles_full.csv"):
    if not data:
        print("No data to save.")
        return
    df = pd.DataFrame(data)
    df = df[["Title", "Url", "Date", "Description"]]
    df.to_csv(filename, index=False, encoding="utf-8-sig")
    print(f"✅ Saved {len(df)} articles to {filename}")

In [9]:
def run_scraper():
    driver = init_driver()
    all_links = set()
    try:
        for section in SECTIONS:
            links = extract_article_links_dynamic(driver, section)
            print(f"✅ {len(links)} articles from {section}")
            all_links.update(links)

        print(f"🔗 Total unique articles collected: {len(all_links)}")

        data = []
        for i, url in enumerate(all_links, 1):
            print(f"[{i}/{len(all_links)}] Scraping: {url}")
            article = extract_article_data(driver, url)
            data.append(article)

        save_to_csv(data)

    finally:
        driver.quit()

In [10]:
run_scraper()

🔍 Visiting: https://www.aljazeera.com/news
✅ 72 articles from https://www.aljazeera.com/news
🔍 Visiting: https://www.aljazeera.com/middle-east
✅ 648 articles from https://www.aljazeera.com/middle-east
🔍 Visiting: https://www.aljazeera.com/africa
✅ 19 articles from https://www.aljazeera.com/africa
🔍 Visiting: https://www.aljazeera.com/asia
✅ 448 articles from https://www.aljazeera.com/asia
🔍 Visiting: https://www.aljazeera.com/europe
✅ 67 articles from https://www.aljazeera.com/europe
🔍 Visiting: https://www.aljazeera.com/us-canada
✅ 42 articles from https://www.aljazeera.com/us-canada
🔍 Visiting: https://www.aljazeera.com/business-economy
✅ 0 articles from https://www.aljazeera.com/business-economy
🔍 Visiting: https://www.aljazeera.com/sports
✅ 105 articles from https://www.aljazeera.com/sports
🔍 Visiting: https://www.aljazeera.com/features
✅ 337 articles from https://www.aljazeera.com/features
🔍 Visiting: https://www.aljazeera.com/opinions
✅ 1029 articles from https://www.aljazeera.co

[59/2491] Scraping: https://www.aljazeera.com/features/2025/7/30/they-chase-ambulances-russias-record-attacks-on-ukraines-healthcare
[60/2491] Scraping: https://www.aljazeera.com/opinions/2024/1/10/can-president-boakai-deliver-war-crimes-justice-in-liberia
[61/2491] Scraping: https://www.aljazeera.com/news/2025/7/22/28-countries-called-for-an-end-to-israels-war-on-gaza-what-did-they-say
[62/2491] Scraping: https://www.aljazeera.com/news/2025/6/20/irans-dual-nationals-caught-between-israeli-bombs-and-family-ties
[63/2491] Scraping: https://www.aljazeera.com/news/2025/7/10/if-trump-wants-gaza-ceasefire-he-must-pressure-netanyahu-experts-say
[64/2491] Scraping: https://www.aljazeera.com/opinions/2024/11/19/on-genocide-trump-will-be-no-different-from-biden
[65/2491] Scraping: https://www.aljazeera.com/opinions/2025/3/17/no-nerds-and-their-technologies-are-not-going-to-save-the-world
[66/2491] Scraping: https://www.aljazeera.com/opinions/2024/5/20/ostracising-israel-at-the-un-should-be-a-pr

[125/2491] Scraping: https://www.aljazeera.com/news/2025/5/11/bangladesh-bans-activities-of-awami-league-the-party-of-ousted-pm-hasina
[126/2491] Scraping: https://www.aljazeera.com/features/2025/5/14/what-does-the-pkks-disarming-mean-for-its-regional-allies
[127/2491] Scraping: https://www.aljazeera.com/news/2025/3/27/trump-says-he-may-cut-china-tariffs-to-secure-tiktok-deal
[128/2491] Scraping: https://www.aljazeera.com/opinions/2024/10/26/how-the-world-keeps-failing-eastern-drc
[129/2491] Scraping: https://www.aljazeera.com/video/newsfeed/2025/6/26/am-i-going-to-die-injured-boy-asks-gaza-paramedic
[130/2491] Scraping: https://www.aljazeera.com/opinions/2024/12/8/what-the-collapse-of-the-syrian-regime-says-about-the-arab-region
[131/2491] Scraping: https://www.aljazeera.com/news/2025/5/11/trump-offers-to-work-with-india-pakistan-on-kashmir
[132/2491] Scraping: https://www.aljazeera.com/opinions/2025/2/1/how-maga-fell-out-with-indian-tech-bros
[133/2491] Scraping: https://www.aljazeer

[191/2491] Scraping: https://www.aljazeera.com/opinions/2025/7/18/israels-narrative-cannot-survive-the-truth-so-its-silencing-the-world
[192/2491] Scraping: https://www.aljazeera.com/opinions/2024/4/16/insurance-firms-should-shun-the-east-african-crude-oil-pipeline
[193/2491] Scraping: https://www.aljazeera.com/opinions/2025/5/24/cooking-in-gaza-is-now-a-toxic-affair
[194/2491] Scraping: https://www.aljazeera.com/opinions/2025/5/26/the-most-dangerous-weapon-in-south-asia-is-not-nuclear
[195/2491] Scraping: https://www.aljazeera.com/news/2025/5/8/malian-juntas-move-to-hold-on-to-power-sparks-rare-revolt
[196/2491] Scraping: https://www.aljazeera.com/opinions/2024/11/3/a-gaza-childs-last-will
[197/2491] Scraping: https://www.aljazeera.com/video/newsfeed/2025/7/23/these-countries-condemn-the-gaza-war-but-have-major-ties-to-israel
[198/2491] Scraping: https://www.aljazeera.com/news/2025/7/29/four-killed-hundreds-arrested-in-ongoing-angola-fuel-hike-protests
[199/2491] Scraping: https://www

[257/2491] Scraping: https://www.aljazeera.com/video/newsfeed/2025/7/30/gazas-starving-children-face-permanent-damage-from-chronic-hunger
[258/2491] Scraping: https://www.aljazeera.com/opinions/2024/11/5/no-state-has-an-inherent-right-to-exist-not-even-israel
[259/2491] Scraping: https://www.aljazeera.com/news/2025/7/23/afp-calls-on-israel-to-allow-evacuation-of-its-journalists-from-gaza
[260/2491] Scraping: https://www.aljazeera.com/opinions/2025/3/30/i-have-finally-understood-the-true-meaning-of-land-day
[261/2491] Scraping: https://www.aljazeera.com/news/2025/7/27/gunfight-in-baghdad-leaves-one-dead-as-paramilitary-group-storms-ministry
[262/2491] Scraping: https://www.aljazeera.com/opinions/2024/6/5/why-indias-election-results-bring-hope-even-in-defeat
[263/2491] Scraping: https://www.aljazeera.com/news/2025/7/22/the-boy-who-bled-to-death-as-an-israeli-soldier-celebrated-his-shot
[264/2491] Scraping: https://www.aljazeera.com/opinions/2023/12/27/the-weapon-of-nuance-in-israels-war-

...

❌ Failed to scrape https://www.aljazeera.com/video/newsfeed/2025/6/26/australian-doctor-calls-on-world-to-wake-up-to-gaza-aid-killings: Message: invalid session id; For documentation on this error, please visit: https://www.selenium.dev/documentation/webdriver/troubleshooting/errors#invalidsessionidexception
Stacktrace:
	GetHandleVerifier [0x0x7ff7ab31e925+77845]
	GetHandleVerifier [0x0x7ff7ab31e980+77936]
	(No symbol) [0x0x7ff7ab0d9b0c]
	(No symbol) [0x0x7ff7ab1208bf]
	(No symbol) [0x0x7ff7ab158792]
	(No symbol) [0x0x7ff7ab153293]
	(No symbol) [0x0x7ff7ab152359]
	(No symbol) [0x0x7ff7ab0a4b05]
	GetHandleVerifier [0x0x7ff7ab5f683d+3059501]
	GetHandleVerifier [0x0x7ff7ab5f0bfd+3035885]
	GetHandleVerifier [0x0x7ff7ab6103f0+3164896]
	GetHandleVerifier [0x0x7ff7ab338c2e+185118]
	GetHandleVerifier [0x0x7ff7ab34053f+216111]
	(No symbol) [0x0x7ff7ab0a3b00]
	GetHandleVerifier [0x0x7ff7ab71bd88+4260984]
	BaseThreadInitThunk [0x0x7ff96adc7374+20]
	RtlUserThreadStart [0x0x7ff96b3fcc91+33]

[1826/

❌ Failed to scrape https://www.aljazeera.com/gallery/2025/3/2/syrias-white-helmets-continue-to-help-people-in-devastated-aleppo: Message: invalid session id; For documentation on this error, please visit: https://www.selenium.dev/documentation/webdriver/troubleshooting/errors#invalidsessionidexception
Stacktrace:
	GetHandleVerifier [0x0x7ff7ab31e925+77845]
	GetHandleVerifier [0x0x7ff7ab31e980+77936]
	(No symbol) [0x0x7ff7ab0d9b0c]
	(No symbol) [0x0x7ff7ab1208bf]
	(No symbol) [0x0x7ff7ab158792]
	(No symbol) [0x0x7ff7ab153293]
	(No symbol) [0x0x7ff7ab152359]
	(No symbol) [0x0x7ff7ab0a4b05]
	GetHandleVerifier [0x0x7ff7ab5f683d+3059501]
	GetHandleVerifier [0x0x7ff7ab5f0bfd+3035885]
	GetHandleVerifier [0x0x7ff7ab6103f0+3164896]
	GetHandleVerifier [0x0x7ff7ab338c2e+185118]
	GetHandleVerifier [0x0x7ff7ab34053f+216111]
	(No symbol) [0x0x7ff7ab0a3b00]
	GetHandleVerifier [0x0x7ff7ab71bd88+4260984]
	BaseThreadInitThunk [0x0x7ff96adc7374+20]
	RtlUserThreadStart [0x0x7ff96b3fcc91+33]

[2038/2491] S

❌ Failed to scrape https://www.aljazeera.com/features/2025/5/4/are-the-houthis-opponents-in-yemen-seeking-us-support: Message: invalid session id; For documentation on this error, please visit: https://www.selenium.dev/documentation/webdriver/troubleshooting/errors#invalidsessionidexception
Stacktrace:
	GetHandleVerifier [0x0x7ff7ab31e925+77845]
	GetHandleVerifier [0x0x7ff7ab31e980+77936]
	(No symbol) [0x0x7ff7ab0d9b0c]
	(No symbol) [0x0x7ff7ab1208bf]
	(No symbol) [0x0x7ff7ab158792]
	(No symbol) [0x0x7ff7ab153293]
	(No symbol) [0x0x7ff7ab152359]
	(No symbol) [0x0x7ff7ab0a4b05]
	GetHandleVerifier [0x0x7ff7ab5f683d+3059501]
	GetHandleVerifier [0x0x7ff7ab5f0bfd+3035885]
	GetHandleVerifier [0x0x7ff7ab6103f0+3164896]
	GetHandleVerifier [0x0x7ff7ab338c2e+185118]
	GetHandleVerifier [0x0x7ff7ab34053f+216111]
	(No symbol) [0x0x7ff7ab0a3b00]
	GetHandleVerifier [0x0x7ff7ab71bd88+4260984]
	BaseThreadInitThunk [0x0x7ff96adc7374+20]
	RtlUserThreadStart [0x0x7ff96b3fcc91+33]

[2261/2491] Scraping: ht

❌ Failed to scrape https://www.aljazeera.com/opinions/2024/5/10/with-measles-on-the-rise-rebuilding-trust-in-vaccines-is-a-must: Message: invalid session id; For documentation on this error, please visit: https://www.selenium.dev/documentation/webdriver/troubleshooting/errors#invalidsessionidexception
Stacktrace:
	GetHandleVerifier [0x0x7ff7ab31e925+77845]
	GetHandleVerifier [0x0x7ff7ab31e980+77936]
	(No symbol) [0x0x7ff7ab0d9b0c]
	(No symbol) [0x0x7ff7ab1208bf]
	(No symbol) [0x0x7ff7ab158792]
	(No symbol) [0x0x7ff7ab153293]
	(No symbol) [0x0x7ff7ab152359]
	(No symbol) [0x0x7ff7ab0a4b05]
	GetHandleVerifier [0x0x7ff7ab5f683d+3059501]
	GetHandleVerifier [0x0x7ff7ab5f0bfd+3035885]
	GetHandleVerifier [0x0x7ff7ab6103f0+3164896]
	GetHandleVerifier [0x0x7ff7ab338c2e+185118]
	GetHandleVerifier [0x0x7ff7ab34053f+216111]
	(No symbol) [0x0x7ff7ab0a3b00]
	GetHandleVerifier [0x0x7ff7ab71bd88+4260984]
	BaseThreadInitThunk [0x0x7ff96adc7374+20]
	RtlUserThreadStart [0x0x7ff96b3fcc91+33]

[2450/2491] S

✅ Saved 2491 articles to aljazeera_articles_full.csv
